# 109 persons
## electrodes : 8, 16, 32, 64

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

In [2]:
pip install mne

Note: you may need to restart the kernel to use updated packages.


In [3]:
no_of_patients=109

In [4]:
train=sorted(glob('files3/*.edf'))
train=train[:no_of_patients]
train

['files3/S001R05.edf',
 'files3/S002R05.edf',
 'files3/S003R05.edf',
 'files3/S004R05.edf',
 'files3/S005R05.edf',
 'files3/S006R05.edf',
 'files3/S007R05.edf',
 'files3/S008R05.edf',
 'files3/S009R05.edf',
 'files3/S010R05.edf',
 'files3/S011R05.edf',
 'files3/S012R05.edf',
 'files3/S013R05.edf',
 'files3/S014R05.edf',
 'files3/S015R05.edf',
 'files3/S016R05.edf',
 'files3/S017R05.edf',
 'files3/S018R05.edf',
 'files3/S019R05.edf',
 'files3/S020R05.edf',
 'files3/S021R05.edf',
 'files3/S022R05.edf',
 'files3/S023R05.edf',
 'files3/S024R05.edf',
 'files3/S025R05.edf',
 'files3/S026R05.edf',
 'files3/S027R05.edf',
 'files3/S028R05.edf',
 'files3/S029R05.edf',
 'files3/S030R05.edf',
 'files3/S031R05.edf',
 'files3/S032R05.edf',
 'files3/S033R05.edf',
 'files3/S034R05.edf',
 'files3/S035R05.edf',
 'files3/S036R05.edf',
 'files3/S037R05.edf',
 'files3/S038R05.edf',
 'files3/S039R05.edf',
 'files3/S040R05.edf',
 'files3/S041R05.edf',
 'files3/S042R05.edf',
 'files3/S043R05.edf',
 'files3/S0

In [5]:
def read_data(i,train_split,valid_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
    
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample(frac=1)
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    idx3=int(valid_split*(len(eeg_df2)))
    idx4=int(valid_split*(len(eeg_df2)))+1
    eeg_df3=eeg_df2.iloc[:idx3]
    eeg_df4=eeg_df2.iloc[idx4:]
    return eeg_df1,eeg_df3,eeg_df4,len(eeg_df1),len(eeg_df3),len(eeg_df4)

    

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
xtemp3=[]
ytemp1=[]
ytemp2=[]
ytemp3=[]
for i in range(no_of_patients):
    xtr,xte,xval,ytr,yte,yval=read_data(train[i],0.8,0.5) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    xtemp3.append(xval)
    ytemp1.append(ytr)
    ytemp2.append(yte)
    ytemp3.append(yval)

In [7]:
xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)
xvalid=pd.concat([xtemp3[i] for i in range(0,len(xtemp3))],ignore_index=True)

In [8]:
ytrain=[]
for i in range(len(ytemp1)):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(len(ytemp2)):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        
yvalid=[]
for i in range(len(ytemp3)):
    for j in range(ytemp3[i-1]):
        yvalid.append(i)     

In [9]:
len(xtrain),len(xtest),len(ytrain),len(ytest)

(1300736, 162483, 1300736, 162483)

In [10]:
print(xtest.iloc[:,-1].values)

[ 1.01e-04  1.30e-05 -1.50e-05 ...  3.30e-05  3.50e-05  6.10e-05]


In [11]:
display(xtrain)

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300731,0.000072,-0.000011,-0.000071,-0.000036,-0.000072,-0.000140,-0.000010,-0.000084,-0.000011,0.000069,...,-0.000043,-0.000045,-0.000065,-0.000008,-0.000069,-0.000070,-0.000060,-0.000086,-0.000032,-0.000018
1300732,0.000089,-0.000020,-0.000062,-0.000023,-0.000065,-0.000138,-0.000017,-0.000092,-0.000008,0.000092,...,-0.000035,-0.000029,-0.000050,0.000013,-0.000054,-0.000053,-0.000045,-0.000076,-0.000018,-0.000003
1300733,0.000117,-0.000005,-0.000050,-0.000007,-0.000048,-0.000110,-0.000011,-0.000076,0.000011,0.000111,...,-0.000024,-0.000016,-0.000044,0.000017,-0.000042,-0.000037,-0.000040,-0.000072,-0.000022,0.000001
1300734,0.000135,0.000005,-0.000045,-0.000012,-0.000046,-0.000168,-0.000014,-0.000069,0.000008,0.000087,...,-0.000019,-0.000029,-0.000057,0.000003,-0.000039,-0.000033,-0.000055,-0.000081,-0.000032,-0.000015


In [12]:
def scale_dataset(dataframe):
    x=dataframe.iloc[:,:-1].values
    y=dataframe.iloc[:,-1].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 0-8

In [13]:
xtrain8=xtrain.iloc[:,:8]
xvalid8=xvalid.iloc[:,:8]
xtest8=xtest.iloc[:,:8]

In [14]:
xtrain8['id']=ytrain
xtest8['id']=ytest
xvalid8['id']=yvalid
display(xtrain)

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/394288547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain8['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/394288547.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest8['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/394288547.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300731,0.000072,-0.000011,-0.000071,-0.000036,-0.000072,-0.000140,-0.000010,-0.000084,-0.000011,0.000069,...,-0.000043,-0.000045,-0.000065,-0.000008,-0.000069,-0.000070,-0.000060,-0.000086,-0.000032,-0.000018
1300732,0.000089,-0.000020,-0.000062,-0.000023,-0.000065,-0.000138,-0.000017,-0.000092,-0.000008,0.000092,...,-0.000035,-0.000029,-0.000050,0.000013,-0.000054,-0.000053,-0.000045,-0.000076,-0.000018,-0.000003
1300733,0.000117,-0.000005,-0.000050,-0.000007,-0.000048,-0.000110,-0.000011,-0.000076,0.000011,0.000111,...,-0.000024,-0.000016,-0.000044,0.000017,-0.000042,-0.000037,-0.000040,-0.000072,-0.000022,0.000001
1300734,0.000135,0.000005,-0.000045,-0.000012,-0.000046,-0.000168,-0.000014,-0.000069,0.000008,0.000087,...,-0.000019,-0.000029,-0.000057,0.000003,-0.000039,-0.000033,-0.000055,-0.000081,-0.000032,-0.000015


In [15]:
x8,y8=scale_dataset(xtrain8)
xt8,yt8=scale_dataset(xtest8)
xv8,yv8=scale_dataset(xvalid8)

In [16]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model = Sequential(
    [
        Dense(8, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [17]:
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model.fit(
    x8,y8,epochs=10,validation_data=(xv8,yv8)
)

Epoch 1/10
40648/40648 [==============================] - 159s 4ms/step - loss: 3.0647 - val_loss: 3.1249
Epoch 2/10
40648/40648 [==============================] - 207s 5ms/step - loss: 2.5264 - val_loss: 3.0903
Epoch 3/10
40648/40648 [==============================] - 105s 3ms/step - loss: 2.4193 - val_loss: 3.0858
Epoch 4/10
40648/40648 [==============================] - 105s 3ms/step - loss: 2.3703 - val_loss: 3.0409
Epoch 5/10
40648/40648 [==============================] - 144s 4ms/step - loss: 2.3401 - val_loss: 3.0271
Epoch 6/10
40648/40648 [==============================] - 35s 853us/step - loss: 2.3186 - val_loss: 3.0674
Epoch 7/10
40648/40648 [==============================] - 1108s 27ms/step - loss: 2.3038 - val_loss: 3.0809
Epoch 8/10
40648/40648 [==============================] - 103s 3ms/step - loss: 2.2910 - val_loss: 3.0521
Epoch 9/10
40648/40648 [==============================] - 290s 7ms/step - loss: 2.2801 - val_loss: 3.0636
Epoch 10/10
40648/40648 [==================

In [18]:
y_pred_classes = np.argmax(tf.nn.softmax(model.predict(xt8)).numpy(),axis=1)
print(classification_report(yt8,y_pred_classes))

5078/5078 [==============================] - 2s 389us/step
              precision    recall  f1-score   support

           0       0.08      0.03      0.04      1499
           1       0.09      0.07      0.08      1499
           2       0.23      0.26      0.25      1499
           3       0.33      0.55      0.42      1499
           4       0.55      0.69      0.61      1499
           5       0.33      0.60      0.43      1499
           6       0.37      0.52      0.44      1499
           7       0.47      0.47      0.47      1499
           8       0.27      0.41      0.33      1499
           9       0.30      0.26      0.28      1499
          10       0.35      0.41      0.38      1499
          11       0.29      0.56      0.38      1499
          12       0.28      0.19      0.23      1499
          13       0.15      0.11      0.13      1499
          14       0.08      0.09      0.09      1499
          15       0.43      0.73      0.54      1499
          16       0.0

## 0-16

In [19]:
xtrain16=xtrain.iloc[:,:16]
xtest16=xtest.iloc[:,:16]
xvalid16=xvalid.iloc[:,:16]

In [20]:
xtrain16['id']=ytrain
xtest16['id']=ytest
xvalid16['id']=yvalid
display(xtrain)

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/4039718790.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain16['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/4039718790.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest16['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/4039718790.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

,Channel_0,Channel_1,Channel_2,Channel_3,Channel_4,Channel_5,Channel_6,Channel_7,Channel_8,Channel_9,...,Channel_54,Channel_55,Channel_56,Channel_57,Channel_58,Channel_59,Channel_60,Channel_61,Channel_62,Channel_63
0,-0.000010,0.000020,0.000007,-0.000018,0.000000,0.000004,0.000060,0.000004,0.000012,0.000006,...,-0.000006,0.000030,0.000025,0.000022,0.000013,-0.000009,0.000020,0.000024,0.000089,0.000068
1,0.000010,0.000050,0.000049,0.000022,0.000038,0.000031,0.000084,0.000025,0.000043,0.000049,...,0.000001,0.000044,0.000031,0.000027,0.000025,0.000012,0.000030,0.000025,0.000083,0.000065
2,0.000017,0.000055,0.000059,0.000030,0.000037,0.000025,0.000079,0.000030,0.000048,0.000053,...,0.000006,0.000048,0.000036,0.000033,0.000034,0.000034,0.000041,0.000034,0.000092,0.000078
3,0.000021,0.000063,0.000066,0.000035,0.000040,0.000029,0.000076,0.000035,0.000055,0.000065,...,0.000029,0.000048,0.000041,0.000050,0.000059,0.000063,0.000045,0.000047,0.000121,0.000086
4,0.000027,0.000072,0.000079,0.000048,0.000052,0.000042,0.000091,0.000043,0.000060,0.000076,...,0.000035,0.000029,0.000022,0.000042,0.000062,0.000071,0.000037,0.000051,0.000143,0.000090
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1300731,0.000072,-0.000011,-0.000071,-0.000036,-0.000072,-0.000140,-0.000010,-0.000084,-0.000011,0.000069,...,-0.000043,-0.000045,-0.000065,-0.000008,-0.000069,-0.000070,-0.000060,-0.000086,-0.000032,-0.000018
1300732,0.000089,-0.000020,-0.000062,-0.000023,-0.000065,-0.000138,-0.000017,-0.000092,-0.000008,0.000092,...,-0.000035,-0.000029,-0.000050,0.000013,-0.000054,-0.000053,-0.000045,-0.000076,-0.000018,-0.000003
1300733,0.000117,-0.000005,-0.000050,-0.000007,-0.000048,-0.000110,-0.000011,-0.000076,0.000011,0.000111,...,-0.000024,-0.000016,-0.000044,0.000017,-0.000042,-0.000037,-0.000040,-0.000072,-0.000022,0.000001
1300734,0.000135,0.000005,-0.000045,-0.000012,-0.000046,-0.000168,-0.000014,-0.000069,0.000008,0.000087,...,-0.000019,-0.000029,-0.000057,0.000003,-0.000039,-0.000033,-0.000055,-0.000081,-0.000032,-0.000015


In [21]:
x16,y16=scale_dataset(xtrain16)
xt16,yt16=scale_dataset(xtest16)
xv16,yv16=scale_dataset(xvalid16)

In [22]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model1 = Sequential(
    [
        Dense(16, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [23]:
model1.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model1.fit(
    x16,y16,epochs=10,validation_data=(xv16,yv16)
)

Epoch 1/10
40648/40648 [==============================] - 38s 919us/step - loss: 2.3443 - val_loss: 2.6282
Epoch 2/10
40648/40648 [==============================] - 41s 1ms/step - loss: 1.6826 - val_loss: 2.6804
Epoch 3/10
40648/40648 [==============================] - 39s 954us/step - loss: 1.5307 - val_loss: 2.6423
Epoch 4/10
40648/40648 [==============================] - 38s 937us/step - loss: 1.4580 - val_loss: 2.6954
Epoch 5/10
40648/40648 [==============================] - 39s 948us/step - loss: 1.4046 - val_loss: 2.6254
Epoch 6/10
40648/40648 [==============================] - 38s 941us/step - loss: 1.3540 - val_loss: 2.6973
Epoch 7/10
40648/40648 [==============================] - 41s 1ms/step - loss: 1.3217 - val_loss: 2.6843
Epoch 8/10
40648/40648 [==============================] - 43s 1ms/step - loss: 1.2960 - val_loss: 2.6449
Epoch 9/10
40648/40648 [==============================] - 44s 1ms/step - loss: 1.2775 - val_loss: 2.6737
Epoch 10/10
40648/40648 [====================

In [24]:
y_pred_classes = np.argmax(tf.nn.softmax(model1.predict(xt16)).numpy(),axis=1)
print(classification_report(yt16,y_pred_classes))

5078/5078 [==============================] - 2s 386us/step
              precision    recall  f1-score   support

           0       0.31      0.17      0.22      1499
           1       0.27      0.16      0.20      1499
           2       0.26      0.24      0.25      1499
           3       0.39      0.34      0.36      1499
           4       0.66      0.84      0.74      1499
           5       0.46      0.73      0.57      1499
           6       0.57      0.93      0.70      1499
           7       0.81      0.55      0.65      1499
           8       0.29      0.38      0.33      1499
           9       0.39      0.29      0.34      1499
          10       0.71      0.31      0.43      1499
          11       0.61      0.65      0.63      1499
          12       0.40      0.60      0.48      1499
          13       0.20      0.34      0.25      1499
          14       0.11      0.18      0.14      1499
          15       0.77      0.74      0.76      1499
          16       0.3

## 0-32

In [25]:
xtrain32=xtrain.iloc[:,:32]
xtest32=xtest.iloc[:,:32]
xvalid32=xvalid.iloc[:,:32]

In [26]:
xtrain32['id']=ytrain
xtest32['id']=ytest
xvalid32['id']=yvalid

/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/1675347936.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtrain32['id']=ytrain
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/1675347936.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  xtest32['id']=ytest
/var/folders/gk/c7h7qkyd3sn7d5qd8rlphgk80000gn/T/ipykernel_63550/1675347936.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [27]:
x32,y32=scale_dataset(xtrain32)
xt32,yt32=scale_dataset(xtest32)
xv32,yv32=scale_dataset(xvalid32)

In [28]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model2 = Sequential(
    [
        Dense(32, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [29]:
model2.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model2.fit(
    x32,y32,epochs=10,validation_data=(xv32,yv32)
)

Epoch 1/10
40648/40648 [==============================] - 45s 1ms/step - loss: 1.5761 - val_loss: 1.9843
Epoch 2/10
40648/40648 [==============================] - 50s 1ms/step - loss: 0.7871 - val_loss: 1.9237
Epoch 3/10
40648/40648 [==============================] - 50s 1ms/step - loss: 0.6469 - val_loss: 1.7940
Epoch 4/10
40648/40648 [==============================] - 50s 1ms/step - loss: 0.5822 - val_loss: 1.8318
Epoch 5/10
40648/40648 [==============================] - 51s 1ms/step - loss: 0.5422 - val_loss: 1.9162
Epoch 6/10
40648/40648 [==============================] - 54s 1ms/step - loss: 0.5145 - val_loss: 1.8053
Epoch 7/10
40648/40648 [==============================] - 55s 1ms/step - loss: 0.4933 - val_loss: 1.8457
Epoch 8/10
40648/40648 [==============================] - 55s 1ms/step - loss: 0.4769 - val_loss: 1.8220
Epoch 9/10
40648/40648 [==============================] - 54s 1ms/step - loss: 0.4623 - val_loss: 1.9139
Epoch 10/10
40648/40648 [==============================

In [30]:
y_pred_classes = np.argmax(tf.nn.softmax(model2.predict(xt32)).numpy(),axis=1)
print(classification_report(yt32,y_pred_classes))

5078/5078 [==============================] - 4s 694us/step
              precision    recall  f1-score   support

           0       0.68      0.52      0.59      1499
           1       0.74      0.22      0.34      1499
           2       0.71      0.66      0.68      1499
           3       0.46      0.36      0.40      1499
           4       0.96      0.93      0.94      1499
           5       0.88      0.66      0.75      1499
           6       0.87      0.91      0.89      1499
           7       0.89      0.58      0.70      1499
           8       0.55      0.62      0.59      1499
           9       0.64      0.62      0.63      1499
          10       0.94      0.91      0.92      1499
          11       0.78      0.96      0.86      1499
          12       0.69      0.80      0.74      1499
          13       0.61      0.72      0.66      1499
          14       0.41      0.66      0.50      1499
          15       0.82      0.95      0.88      1499
          16       0.2

## 0-64

In [31]:
xtrain['id']=ytrain
xtest['id']=ytest
xvalid['id']=yvalid

In [32]:
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)
xv,yv=scale_dataset(xvalid)

In [33]:
tf.random.set_seed(1234)  # applied to achieve consistent results
model3 = Sequential(
    [
        Dense(64, activation = 'relu',   name = "L1"),
        Dense(256, activation = 'relu', name = "L2"),
        Dense(128, activation = 'relu', name = "L3"),
        Dense(64, activation = 'relu',   name = "L4"),
        Dense(32, activation = 'relu', name = "L5"),
        Dense(109, activation = 'linear', name = "L6"),
    ]
)

In [34]:
model3.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(0.001),
)

model3.fit(
    x,y,epochs=10,validation_data=(xv,yv)
)

Epoch 1/10
40648/40648 [==============================] - 65s 2ms/step - loss: 1.2394 - val_loss: 2.1192
Epoch 2/10
40648/40648 [==============================] - 68s 2ms/step - loss: 0.5601 - val_loss: 2.0471
Epoch 3/10
40648/40648 [==============================] - 66s 2ms/step - loss: 0.4091 - val_loss: 1.9937
Epoch 4/10
40648/40648 [==============================] - 68s 2ms/step - loss: 0.3297 - val_loss: 1.7668
Epoch 5/10
40648/40648 [==============================] - 68s 2ms/step - loss: 0.2784 - val_loss: 1.7246
Epoch 6/10
40648/40648 [==============================] - 70s 2ms/step - loss: 0.2448 - val_loss: 1.8357
Epoch 7/10
40648/40648 [==============================] - 79s 2ms/step - loss: 0.2197 - val_loss: 1.5308
Epoch 8/10
40648/40648 [==============================] - 71s 2ms/step - loss: 0.2014 - val_loss: 1.6235
Epoch 9/10
40648/40648 [==============================] - 65s 2ms/step - loss: 0.1856 - val_loss: 1.7192
Epoch 10/10
40648/40648 [==============================

In [35]:
y_pred_classes = np.argmax(tf.nn.softmax(model3.predict(xt)).numpy(),axis=1)
print(classification_report(yt,y_pred_classes))

5078/5078 [==============================] - 2s 401us/step
              precision    recall  f1-score   support

           0       0.83      0.84      0.84      1499
           1       0.94      0.82      0.88      1499
           2       0.78      0.85      0.81      1499
           3       0.79      0.83      0.81      1499
           4       0.99      0.99      0.99      1499
           5       0.96      0.92      0.94      1499
           6       0.92      0.97      0.95      1499
           7       0.96      0.86      0.91      1499
           8       0.74      0.88      0.80      1499
           9       0.75      0.71      0.73      1499
          10       0.87      0.88      0.87      1499
          11       0.88      0.95      0.92      1499
          12       0.81      0.95      0.87      1499
          13       0.82      0.91      0.86      1499
          14       0.52      0.84      0.64      1499
          15       0.99      0.98      0.98      1499
          16       0.9